# NLP - CA6
* **Name:** Mohammad Mahdi Salmani
* **Student id:** 810102174

## Load libraries

In [ ]:
%pip install -r requirements.txt

In [65]:
import os
import json
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import LocalFileStore
from dotenv import load_dotenv
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_together import ChatTogether
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Literal
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from IPython.core.display import Markdown
from langchain_core.messages import HumanMessage, AIMessage
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain.embeddings import CacheBackedEmbeddings

## 1. Load dataset

In [79]:
with open("configs.json") as f:
    configs = json.loads(f.read())

os.environ["TAVILY_API_KEY"] = configs["api"]["TAVILY_API_KEY"]
os.environ["TOGETHER_API_KEY"] = configs["api"]["TOGETHER_API_KEY"]

### Crawl pdf list from site

In [2]:
url = "https://stanford.edu/~jurafsky/slp3/"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

chapters = []
for link in soup.find_all('a', href=True):
    if '.pdf' in link['href'] and link['href'].split('.')[0].isdigit():
        chapters.append(link['href'])

pdf_links = [url+link for link in chapters]
pdf_links

['https://stanford.edu/~jurafsky/slp3/2.pdf',
 'https://stanford.edu/~jurafsky/slp3/3.pdf',
 'https://stanford.edu/~jurafsky/slp3/4.pdf',
 'https://stanford.edu/~jurafsky/slp3/5.pdf',
 'https://stanford.edu/~jurafsky/slp3/6.pdf',
 'https://stanford.edu/~jurafsky/slp3/7.pdf',
 'https://stanford.edu/~jurafsky/slp3/8.pdf',
 'https://stanford.edu/~jurafsky/slp3/9.pdf',
 'https://stanford.edu/~jurafsky/slp3/10.pdf',
 'https://stanford.edu/~jurafsky/slp3/11.pdf',
 'https://stanford.edu/~jurafsky/slp3/13.pdf',
 'https://stanford.edu/~jurafsky/slp3/14.pdf',
 'https://stanford.edu/~jurafsky/slp3/15.pdf',
 'https://stanford.edu/~jurafsky/slp3/16.pdf',
 'https://stanford.edu/~jurafsky/slp3/17.pdf',
 'https://stanford.edu/~jurafsky/slp3/18.pdf',
 'https://stanford.edu/~jurafsky/slp3/19.pdf',
 'https://stanford.edu/~jurafsky/slp3/20.pdf',
 'https://stanford.edu/~jurafsky/slp3/21.pdf',
 'https://stanford.edu/~jurafsky/slp3/22.pdf',
 'https://stanford.edu/~jurafsky/slp3/23.pdf']

### Download PDFs

In [4]:
selected_pdf_links = pdf_links[:10] # Just retrieve from Chapter I: Fundamental Algorithms

In [12]:
documents = []
for pdf_link in tqdm(selected_pdf_links):
    loader = PyPDFLoader(pdf_link)
    documents.extend(loader.load())

100%|██████████| 10/10 [02:42<00:00, 16.28s/it]


In [13]:
len(documents)

275

### Chunking

In [14]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
chunks = text_splitter.split_documents(documents)

len(chunks)

904

### Vector Store

In [19]:
%pip install --upgrade --quiet  langchain-openai faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [18]:
embedding_function = HuggingFaceEmbeddings(show_progress=True, multi_process=True)

model.safetensors:   2%|2         | 10.5M/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [24]:
store = LocalFileStore("./embedding_cache/")
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedding_function, store, namespace=embedding_function.model_name)

### Retriever

In [30]:
# %%time
vector_store = FAISS.from_documents(documents=chunks, embedding=cached_embedder)

CPU times: total: 203 ms
Wall time: 2min 34s


In [57]:
query = "Recurrent Neural Networks"
results = vector_store.similarity_search_with_relevance_scores(query, k=3)

for i, result in enumerate(results):
    print(f'[{i+1}] Score: {result[1]:.3f}, Metadata: {result[0].metadata}')
    print(f'\n{result[0].page_content[:200]}...\n')

[1] Score: 0.589, Metadata: {'source': 'https://stanford.edu/~jurafsky/slp3/9.pdf', 'page': 0}

the prior context, in its recurrent connections , allowing the model’s decision to
depend on information from hundreds of words in the past. We’ll see how to apply
the model to the task of language mo...

[2] Score: 0.562, Metadata: {'source': 'https://stanford.edu/~jurafsky/slp3/9.pdf', 'page': 23}

covered:
• In simple Recurrent Neural Networks sequences are processed one element at
a time, with the output of each neural unit at time tbased both on the current
input at tand the hidden layer from...

[3] Score: 0.553, Metadata: {'source': 'https://stanford.edu/~jurafsky/slp3/9.pdf', 'page': 26}

neural networks. SemEval-2016 .
Gers, F. A., J. Schmidhuber, and F. Cummins. 2000. Learn-
ing to forget: Continual prediction with lstm. Neural
computation , 12(10):2451–2471.
Giles, C. L., G. M. Kuhn...



In [58]:
sample_queries = [
    "What is the advantage of Bidirectional RNNs?",
    "What is the differences between a stack and a queue.",
    "Who is the president of United States?"
]

In [60]:
faiss_retriever = vector_store.as_retriever(search_kwargs={"k": 3})

for i, question in enumerate(sample_queries):
    print(f'[Q{i+1}] {question}\n')
    results = faiss_retriever.invoke(query)
    for j, result in enumerate(results):
        print(f'Retrieve {j+1}) {result.metadata}')
        print(f'\n{result.page_content[:200]}...\n')

[Q1] What is the advantage of Bidirectional RNNs?

Retrieve 1) {'source': 'https://stanford.edu/~jurafsky/slp3/9.pdf', 'page': 0}

the prior context, in its recurrent connections , allowing the model’s decision to
depend on information from hundreds of words in the past. We’ll see how to apply
the model to the task of language mo...

Retrieve 2) {'source': 'https://stanford.edu/~jurafsky/slp3/9.pdf', 'page': 23}

covered:
• In simple Recurrent Neural Networks sequences are processed one element at
a time, with the output of each neural unit at time tbased both on the current
input at tand the hidden layer from...

Retrieve 3) {'source': 'https://stanford.edu/~jurafsky/slp3/9.pdf', 'page': 26}

neural networks. SemEval-2016 .
Gers, F. A., J. Schmidhuber, and F. Cummins. 2000. Learn-
ing to forget: Continual prediction with lstm. Neural
computation , 12(10):2451–2471.
Giles, C. L., G. M. Kuhn...

[Q2] What is the differences between a stack and a queue.

Retrieve 1) {'source': 'https://s

## Part3: EnsembleRetriever

In [ ]:
%pip install --upgrade --quiet  rank_bm25

In [68]:
bm25_retriever = BM25Retriever.from_documents(documents=chunks, embedding=cached_embedder)

In [72]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.4, 0.6]
)

In [73]:
ensemble_retriever.invoke(sample_queries[0])

[Document(page_content='sequence from tto the end of the sequence.\nAbidirectional RNN (Schuster and Paliwal, 1997) combines two independentbidirectional\nRNN\nRNNs, one where the input is processed from the start to the end, and the other from\nthe end to the start. We then concatenate the two representations computed by the\nnetworks into a single vector that captures both the left and right contexts of an input\nat each point in time. Here we use either the semicolon ”;” or the equivalent symbol\n⊕to mean vector concatenation:\nht= [hf\nt;hb\nt]\n=hf\nt⊕hb\nt (9.18)\nFig. 9.11 illustrates such a bidirectional network that concatenates the outputs of\nthe forward and backward pass. Other simple ways to combine the forward and\nbackward contexts include element-wise addition or multiplication. The output at\neach step in time thus captures information to the left and to the right of the current\ninput. In sequence labeling applications, these concatenated outputs can serve as the\nbas

## Part4: